In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import json

import IPython.display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.interpolate
import scipy.signal

import pymedphys
import pymedphys._wlutz.bbpredict
import pymedphys._wlutz.pylinac
import pymedphys._wlutz.iview
import pymedphys._wlutz.imginterp
import pymedphys._wlutz.findfield
import pymedphys._wlutz.findbb
import pymedphys._wlutz.reporting

In [ ]:
intput_dir = pathlib.Path(r'S:\DataExchange\SASH\iViewDB_decoded\indexed')
output_dir = pathlib.Path(r'S:\DataExchange\SASH\iViewDB_decoded\indexed\results')

output_dir.mkdir(exist_ok=True)

In [ ]:
penumbra = 2
edge_lengths = [30, 30]
initial_rotation = 0
bb_diameter = 8

pd.set_option("display.max_rows", 101)

In [ ]:
frame_paths_list = list(intput_dir.glob('*.png'))

In [ ]:
frame_number = [
    int(path.stem.split('_')[1]) for path in frame_paths_list
]



sorted_paths = np.array(frame_paths_list)[np.argsort(frame_number)]

In [ ]:
# def plot_pylinac_comparison(field, bb_diameter, edge_lengths, penumbra, field_centre, field_rotation, pylinac):
#     bb_centre = pymedphys._wlutz.findbb.optimise_bb_centre(
#         field, bb_diameter, edge_lengths, penumbra, field_centre, field_rotation, pylinac_tol=np.inf
#     )
    
#     fig = pymedphys._wlutz.reporting.image_analysis_figure(
#         x,
#         y,
#         img,
#         bb_centre,
#         field_centre,
#         field_rotation,
#         bb_diameter,
#         edge_lengths,
#         penumbra,
#     )

#     plt.title('PyMedPhys Basinhopping Method')
    
    
#     plt.show()

In [ ]:
import imageio

def iview_image_transform(image_path):
    img = imageio.imread(image_path)
    if np.shape(img) != (512, 512):
        raise ValueError(
            f"Expect iView images to be 512x512 pixels\nShhape = {np.shape(img)}"
        )
    img = img[:, 1:-1]

    if img.dtype != np.dtype("uint16"):
        raise ValueError("Expect iView images to have a pixel type of unsigned 16 bit")
    img = 1 - img[::-1, :] / 2 ** 16

    shape = np.shape(img)
    x = np.arange(-shape[1] / 2, shape[1] / 2) / 2
    y = np.arange(-shape[0] / 2, shape[0] / 2) / 2

    return x, y, img

In [ ]:
field_centres = []
bb_centres = []



for image_path in sorted_paths:
    print(image_path)
    x, y, img = iview_image_transform(image_path)

    field = pymedphys._wlutz.imginterp.create_interpolated_field(x, y, img)
    initial_centre = pymedphys._wlutz.findfield._initial_centre(x, y, img)
    
    try:
        field_centre, field_rotation = pymedphys._wlutz.findfield.field_centre_and_rotation_refining(
            field, edge_lengths, penumbra, initial_centre, initial_rotation=initial_rotation
        )            

        bb_centre = pymedphys._wlutz.findbb.optimise_bb_centre(
            field, bb_diameter, edge_lengths, penumbra, field_centre, field_rotation
        )
        
        field_centres.append(field_centre)
        bb_centres.append(bb_centre)
    except Exception as e:
        print(e)
    
    

In [ ]:
field_centres = np.array(field_centres)
bb_centres = np.array(bb_centres)

In [ ]:
deviations = field_centres - bb_centres

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(deviations[:,0])
plt.xlabel('Frame number')
plt.ylabel('Transverse Field position - BB position (mm)')

In [ ]:
np.max(deviations) - np.min(deviations)